In [1]:
import pandas as pd
from difflib import get_close_matches
import re
import json
import copy

In [99]:
class University:
    def __init__(self, name, eng_name, country):
        self.name = name
        self.eng_name = eng_name
        self.country = country
        self.arr = [0,0,0,0,0,0,0,0]
        self.data_arr = [[], [], [], [], [], [], [], []]
    
    def add(self, data, flag_cum, flag_our):
        flag = 0
        flag += flag_cum * 4
        flag += flag_our * 2
        if(data.性別 == 'F'):
            flag += 1
        self.data_arr[flag].append(data)
        self.arr[flag] += 1
        
    def output(self):
        return [self.name, self.eng_name], self.country, self.arr

In [100]:
class Table6:
    def __init__(self, configure):
        self.data = {}
        self.config = configure
        pass
    
    def add(self, data, flag_cum, flag_our):
        edu_name_ch = data.合作學校中文名稱
        edu_name_eng = data.合作學校英文名稱
        country = data.合作學校國別
        if not edu_name_ch in self.data:
            self.data[edu_name_ch] = University(edu_name_ch, edu_name_eng, country)
        self.data[edu_name_ch].add(data, flag_cum, flag_our)
    
    def output(self):
        values = []
        for key, item in self.data.items():
            print(item.arr)
            self.config.values[11:13], self.config.values[10], self.config.values[14:22] = item.output()
            values.append(copy.copy(self.config))
        return values

In [101]:
data1 = pd.read_excel("./11003期校庫資料(給校資組).xlsx", sheet_name="學六來校新增")
data2 = pd.read_excel("./11003期校庫資料(給校資組).xlsx", sheet_name="學六赴外新增")
cum_data1 = pd.read_excel("./11003期校庫資料(給校資組).xlsx", sheet_name="(學6)來校雙聯學位生(修讀 30 畢業79)")
cum_data2 = pd.read_excel("./11003期校庫資料(給校資組).xlsx", sheet_name="(學6)赴外雙聯學位生(修讀 28 畢業188)")
config = pd.read_excel("./第11003期_學6.雙聯學制學生人數統計表(3月、10月填報).xls")

In [102]:
phd_depts = list(set(config[config["學制班別"] == "博士班"].單位))
master_depts = list(set(config[config["學制班別"] == "碩士班"].單位))
master_depts2 = list(set(config[config["學制班別"] == "碩士在職專班"].單位))
undergraduate_depts = list(set(config[config["學制班別"] == "學士班"].單位))

EDU_SYS_MAPPING = {
    "博士班" : phd_depts,
    "碩士班" : master_depts,
    "碩士在職專班" : master_depts2,
    "學士班" : undergraduate_depts
}

In [117]:
# Create data structure
objs = {}
for i in range(len(config.values)):
    sys = config.loc[i].學制班別
    dept = config.loc[i].單位
    
    if(not sys in objs):
        objs[sys] = {}
    
    
    if(not dept in objs[sys]):
        objs[sys][dept] = Table6(config.loc[i])

In [118]:
def mapping(dataframe, maps):
    for i in range(len(dataframe.values)):
        sys = dataframe.loc[i].學制班別
        dept = dataframe.loc[i].系所
        if(re.search(r'班', sys) == None):
            sys += '班'
        
        try:
            dept2 = get_close_matches(dept, EDU_SYS_MAPPING[sys])
            maps[dept] = dept2[0]
        except IndexError as e:
            print("Error dept name : ", dept)
    return maps

In [119]:
def classify_data(objs, dataframe,maps, flag_cum, flag_their):
    # classify data
    for i in range(len(dataframe.values)):
        sys = dataframe.loc[i].學制班別
        dept = dataframe.loc[i].系所
        if(re.search(r'班', sys) == None):
            sys += '班'

        dept = maps[dept]
        objs[sys][dept].add(dataframe.loc[i], flag_cum, flag_their)

In [69]:
maps = {}
maps = mapping(data1, maps)
maps = mapping(data2, maps)
maps = mapping(cum_data1, maps)
maps = mapping(cum_data2, maps)
with open("dept_maps.json", 'w') as file:
    text = json.dumps(maps, indent=4, ensure_ascii=False)
    file.write(text)

Error dept name :  數學系應用數學博士班
Error dept name :  數學系應用數學博士班


In [120]:
with open("dept_maps.json", "r") as file:
    maps = json.load(file)
classify_data(objs, data1, maps, flag_cum=0, flag_their=1) # 學六來校新增
classify_data(objs, data2,maps, 0, 0) # 學六赴外新增
classify_data(objs, cum_data1, maps, flag_cum=1, flag_their=1) # (學6)來校 累積
classify_data(objs, cum_data2, maps, flag_cum=1, flag_their=0) # (學6)赴外 累積

In [121]:
# output
result = []
resultDf = pd.DataFrame()

for key in objs:
    for key2 in objs[key]:
        out = objs[key][key2].output()
        if(len(out) != 0):
            result.append(out)
            
resultDf = []
for i in range(len(result)):
    for j in range(len(result[i])):
        resultDf.append(result[i][j])

resultDf = pd.DataFrame(resultDf)
resultDf.to_excel("output.xlsx")

[0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 15, 2]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 2, 0, 0, 0, 1]
[0, 0, 0, 0, 20, 1, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 2, 3, 0, 0]
[0, 0, 0, 0, 2, 2, 0, 0]
[0, 0, 0, 0, 2, 2, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 5, 0, 2, 1]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 2, 4, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0]
[1, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 2, 6, 0, 0]
[0, 0, 0, 0, 0, 2, 0, 0]
[0, 0, 0, 0, 3, 1, 0, 0]
[0, 1, 0, 0, 1, 1, 0, 0

In [98]:
len(objs["碩士班"]["電機工程學系"].data["布蘭登堡科技大學"].data_arr[4])

5

In [111]:
objs["碩士班"]["國際經營管理研究所(IMBA)"].data["諾曼第管理學院"].data_arr[0]

[編號                                                                 20
 學院                                                               管理學院
 系所                                                    國際經營管理研究所(IMBA)
 學制班別                                                              碩士班
 學號                                                          RA6097175
 姓名                                                                秦艾羅
 英文姓名                                         Regruto Jean-Loup Axel  
 性別                                                                  M
 國籍                                                                 法國
 合作學校國別                                                             法國
 合作學校中文名稱                                                      諾曼第管理學院
 合作學校英文名稱                             Ecole de Management de Normandie
 入學學年                                                            10902
 畢業學年                                                              NaN
 是否已畢業